In [1]:
import pandas as pd
import numpy as np
import os
import sys

sys.path.append(os.path.join(os.path.dirname(os.getcwd())))
from interfaces.alchemy import AlchemyInterface
import configparser
from db.schema import StationLocation, Station, Messages
from geopy.distance import great_circle
import re

In [2]:
c_parser = configparser.ConfigParser()
c_parser.read("../../config.ini")
config = {"db": dict(c_parser["db"])}

In [3]:
target = "W6HBR"

In [4]:
alchemy_interface = AlchemyInterface(config)

In [5]:
station_info = alchemy_interface.select_obj(
    Station, "*", df=True, **{"station_id": target}
)
station_locations = alchemy_interface.select_obj(
    StationLocation, "*", df=True, **{"station": target}
)
station_messages_src = alchemy_interface.select_obj(
    Messages, "*", df=True, **{"src_station": target}
)
station_messages_dst = alchemy_interface.select_obj(
    Messages, "*", df=True, **{"dst_station": target}
)

In [6]:
station_info.ssid.describe()

count     1
unique    1
top       2
freq      1
Name: ssid, dtype: object

In [ ]:
def analyze_timestamps(timestamps):
    # Convertir los timestamps a un objeto DateTime
    timestamps = pd.to_datetime(timestamps)

    # Calcular la frecuencia de los timestamps
    time_diffs = timestamps.diff()  # Diferencia entre cada timestamp
    median_freq = time_diffs.median()
    std_diff = time_diffs.std()

    # Reajustar el índice de time_diffs para alinear con timestamps
    time_diffs = time_diffs.reindex(timestamps.index, method="ffill")

    max_accepted_gap = median_freq + std_diff * 2
    gaps = timestamps[time_diffs > max_accepted_gap]

    # Información adicional
    mean_freq = time_diffs.mean()
    min_freq = time_diffs.min()
    max_freq = time_diffs.max()
    start_date = timestamps.min()
    end_date = timestamps.max()
    num_timestamps = len(timestamps)
    total_duration = end_date - start_date

    # Crear un diccionario con los resultados
    analysis_results = {
        "mean_frequency": mean_freq,
        "median_frequency": median_freq,
        "min_frequency": min_freq,
        "max_frequency": max_freq,
        "start_date": start_date,
        "end_date": end_date,
        "gaps": gaps,
        "num_timestamps": num_timestamps,
        "recorded_time": total_duration,
    }

    return analysis_results

In [ ]:
analyze_timestamps(station_locations["timestamp"])

In [ ]:
def analyze_locations(locations):
    # Convertir las ubicaciones a un formato numpy para un fácil manejo
    locations_np = np.array(locations)

    # Calcular la distancia total y la distancia promedio entre puntos
    distances = [
        great_circle(locations_np[i], locations_np[i + 1]).kilometers
        for i in range(len(locations_np) - 1)
    ]
    total_distance = sum(distances)
    average_distance = np.mean(distances) if distances else 0
    std_dev_distance = np.std(distances) if distances else 0

    # Función para calcular el punto medio
    def calculate_midpoint(locs):
        lat = locs[:, 0]
        lon = locs[:, 1]
        return np.mean(lat), np.mean(lon)

    # Calcular el punto medio
    midpoint = calculate_midpoint(locations_np)

    # Encontrar los extremos geográficos
    northernmost = max(locations, key=lambda x: x[0])
    southernmost = min(locations, key=lambda x: x[0])
    easternmost = max(locations, key=lambda x: x[1])
    westernmost = min(locations, key=lambda x: x[1])

    # Calcular la distancia máxima desde el punto medio
    max_distance_from_mid = max(
        great_circle(midpoint, loc).kilometers for loc in locations
    )

    # Calcular el área aproximada cubierta por los puntos
    latitudes = locations_np[:, 0]
    longitudes = locations_np[:, 1]
    approx_area = (
        great_circle(
            (latitudes.min(), longitudes.min()), (latitudes.min(), longitudes.max())
        ).kilometers
        * great_circle(
            (latitudes.min(), longitudes.min()), (latitudes.max(), longitudes.min())
        ).kilometers
    )

    return {
        "locations": locations,
        "total_distance_km": total_distance,
        "average_distance_km": average_distance,
        "std_dev_distance_km": std_dev_distance,
        "midpoint": midpoint,
        "northernmost": northernmost,
        "southernmost": southernmost,
        "easternmost": easternmost,
        "westernmost": westernmost,
        "max_distance_from_midpoint_km": max_distance_from_mid,
        "approximate_area_sq_km": approx_area,
    }

In [ ]:
loc = analyze_locations(station_locations[["latitude", "longitude"]].values.tolist())

In [ ]:
import plotly.graph_objects as go


def plot_map(analysis):
    # Extraer información del análisis
    midpoint = analysis["midpoint"]
    northernmost = analysis["northernmost"]
    southernmost = analysis["southernmost"]
    easternmost = analysis["easternmost"]
    westernmost = analysis["westernmost"]

    # Coordenadas para la bounding box
    lon_box = [
        westernmost[1],
        easternmost[1],
        easternmost[1],
        westernmost[1],
        westernmost[1],
    ]
    lat_box = [
        northernmost[0],
        northernmost[0],
        southernmost[0],
        southernmost[0],
        northernmost[0],
    ]

    # Crear figura
    fig = go.Figure()

    # Añadir bounding box como un camino cerrado
    fig.add_trace(
        go.Scattermapbox(
            lon=lon_box,
            lat=lat_box,
            mode="lines",
            line=go.scattermapbox.Line(color="grey"),
            name="Bounding Box",
        )
    )

    # Añadir puntos extremos y punto medio
    for point, name in zip(
        [northernmost, southernmost, easternmost, westernmost, midpoint],
        ["Northernmost", "Southernmost", "Easternmost", "Westernmost", "Midpoint"],
    ):
        fig.add_trace(
            go.Scattermapbox(
                lon=[point[1]],
                lat=[point[0]],
                mode="markers",
                marker=go.scattermapbox.Marker(size=5, color="black"),
            )
        )

    # Establecer la vista inicial del mapa
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox={"center": {"lon": midpoint[1], "lat": midpoint[0]}, "zoom": 10},
    )
    lats, lons = zip(*analysis["locations"])
    fig.add_trace(
        go.Scattermapbox(
            lon=lons,
            lat=lats,
            mode="markers+lines",
            line=go.scattermapbox.Line(color="black"),
            marker=go.scattermapbox.Marker(size=10, color="blue"),
            name="Bounding Box",
        )
    )

    # Mostrar figura
    fig.show()

In [ ]:
def analyze_loc_temporal(locations_timestamp):
    # Convert column to datetime
    locations_timestamp["timestamp"].apply(lambda x: pd.to_datetime(x))
    coordinates = list(
        zip(locations_timestamp["latitude"], locations_timestamp["longitude"])
    )
    locations_timestamp = locations_timestamp.assign(coordinate=coordinates)
    locations_timestamp["time_elapsed"] = (
        locations_timestamp.groupby("coordinate")["timestamp"]
        .diff()
        .fillna(pd.Timedelta(seconds=0))
    )

    # Calcular el tiempo total pasado en cada coordenada
    total_time_elapsed = locations_timestamp.groupby("coordinate")["time_elapsed"].sum()

    # Calcular la frecuencia de visitas a cada coordenada
    visit_frequency = locations_timestamp["coordinate"].value_counts()

    # Calcular la fecha y hora de la primera y última visita a cada coordenada
    first_visit = locations_timestamp.groupby("coordinate")["timestamp"].min()
    last_visit = locations_timestamp.groupby("coordinate")["timestamp"].max()

    # Calcular la duración total de la estancia en cada coordenada
    total_stay_duration = last_visit - first_visit

    # Crear un DataFrame con los resultados
    results_df = pd.DataFrame(
        {
            "total_time_elapsed": total_time_elapsed,
            "visit_frequency": visit_frequency,
            "first_visit": first_visit,
            "last_visit": last_visit,
            "total_stay_duration": total_stay_duration,
        }
    )

    return results_df

In [ ]:
analyze_loc_temporal(station_locations[["latitude", "longitude", "timestamp"]])

In [100]:
def analyze_comment(comments):
    # Calcular la frecuencia de comentarios distintos
    unique_comments_freq = comments.value_counts()

    pattern = re.compile(
        r"\b((https?|ftp):\/\/)?([\w-]+(\.[\w-]+)+)([\/\w-]*)*(\?\w+=\w+(&\w+=\w+)*)?\b"
    )
    urls = comments.apply(lambda x: pattern.search(x).group(0))
    has_url = urls.apply(lambda x: x is not None)

    results_list = []
    for comment, freq, has_url_val, url in zip(
        unique_comments_freq.index, unique_comments_freq.values, has_url, urls
    ):
        result_dict = {
            "Comment": comment,
            "Freq": freq,
            "has_url": has_url_val,
            "URL": url,
        }
        results_list.append(result_dict)

    return results_list

In [101]:
analyze_comment(station_messages_src["comment"])

[{'Comment': 'HBRACES.NET, AllStar 49372 (2m) & 49374 (70cm)',
  'Freq': 29,
  'has_url': True,
  'URL': 'HBRACES.NET'}]